Тренировка модели RandomForestClassifier указанной в примерах библиотеки на трафике,
полученном в пункте 4. В качестве данных для тестирования использовать трафик из
пунктов 1-3. Данный модуль должен быть реализован отдельно

In [32]:
from nfstream import NFStreamer
from sklearn.ensemble import RandomForestClassifier
import pickle

nfs = NFStreamer(source='data/4/final.pcapng').to_pandas()
X_train = nfs[["src_port", "bidirectional_packets", "bidirectional_bytes", "bidirectional_duration_ms"]]
Y_train = nfs['application_name'].map({'WireGuard':1, 'IPsec.Azure':2, 'OpenVPN.Azure':3}).fillna(0)

model = RandomForestClassifier()
model.fit(X_train, Y_train)

with open("model.pkl", "wb") as file:
    pickle.dump(model, file)

In [42]:
def print_result(pcap_file:str, name:str):
    print(name)
    nfs = NFStreamer(source=pcap_file).to_pandas()
    X_test = nfs[["src_port", "bidirectional_packets", "bidirectional_bytes", "bidirectional_duration_ms"]]
    Y_test = model.predict(X_test)

    if 1 in Y_test:
        print('     WireGuard used')
    if 2 in Y_test:
        print('     IPSec used')
    if 1 in Y_test:
        print('     OpenVPN used')

    if 1 not in Y_test and 2 not in Y_test and 3 not in Y_test:
        print('     No VPN used')

In [43]:
print_result('data/1/wireguard.pcapng', '1_wireguard')
print_result('data/1/ipsec.pcapng', '1_ipsec')
print_result('data/1/openvpn.pcapng', '1_openvpn')

print_result('data/2/wireguard.pcapng', '2_wireguard')
print_result('data/2/ipsec.pcapng', '2_ipsec')
print_result('data/2/openvpn.pcapng', '2_openvpn')

print_result('data/3/novpn.pcapng', '3_novpn')

1_wireguard
     IPSec used
1_ipsec
     No VPN used
1_openvpn
     No VPN used
2_wireguard
     WireGuard used
     OpenVPN used
2_ipsec
     IPSec used
2_openvpn
     No VPN used
3_novpn
     WireGuard used
     OpenVPN used
